In [ ]:
#import general libraries
import os
import numpy as np

#import serial to be able to directly communicate with teensy if desired
import serial as ser

#import camera library
import PySpin

import sys as sys
sys.path.insert(0,'utils/')
import vidtools as vt

import time as tm


In [ ]:
#define arduino port to connect to.
arduino_name ='COM4'

#initialize camera variables.

frame_rate = 20 #hz.
exposure_time = 400 #us
gain = 1 #multiplicative (AU)
length =5 #seconds
data_type ='uint16'

pixel_bin_size = 4 #binning for both x and y (1 means no binning).

#define path and name for output videos
output_path = '../Data/Test Videos/20240209/Test_datatype/'
sample = 'uint8_test'
prefix = str(length) + 's_' + str(frame_rate) + 'Hz_' + str(exposure_time) + 'us_' + str(pixel_bin_size) + 'xbin_'+str(sample)
barcode = '00'

#define codec to use (if any)
fourcc_code = 0 #0 means no compression. #cv.VideoWriter_fourcc(*'mp4v') for mp4.
ext = '.avi' #needs to match fourcc_code. #'.mp4'


In [ ]:
#start camera
if not vt.detect_cams():  # make sure there are blackfly cameras attached to the computer
 sys.exit()

system = PySpin.System.GetInstance() #start the communication to the cameras

cam_list=system.GetCameras() #get the handles of the cameras

cam=cam_list[0] #set a handle for a specific camera

cam.Init() #initialze the camera

#make sure the camera isn't streaming or some fields won't be writable
try:
    cam.EndAcquisition()
except:
    None

#get camera node map
node_map = cam.GetTLStreamNodeMap()

#set gain
cam.GainAuto.SetValue(PySpin.GainAuto_Off)
cam.Gain.SetValue(gain)

#set exposure
cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Off) # turn off auto exposure
cam.ExposureTime.SetValue(exposure_time) #set the exposure time on the camera

#turn off automatic grey value
cam.AutoExposureTargetGreyValueAuto.SetValue(False)

#turn off automatic gamma correction
cam.GammaEnable.SetValue(False)

#turn off auto sharpness
genable = PySpin.CBooleanPtr(node_map.GetNode("SharpenessAuto"))
if PySpin.IsWritable(genable):
    genable.SetValue(0)

#set framerate
cam.AcquisitionFrameRateEnable.SetValue(True)

if frame_rate > cam.AcquisitionFrameRate.GetMax(): #make sure it doesn't exceed the max
    frame_rate = cam.AcquisitionFrameRate.GetMax()
    print('frame rate exceeds max. Setting to: ' + str(frame_rate))

cam.AcquisitionFrameRate.SetValue(frame_rate)

#bin pixels and reset field of view.
cam.BinningHorizontal.SetValue(pixel_bin_size)
cam.BinningVertical.SetValue(pixel_bin_size)

cam.OffsetX.SetValue(0)
cam.OffsetY.SetValue(0)
cam.Height.SetValue(cam.Height.GetMax())
cam.Width.SetValue(cam.Width.GetMax())

#set bit depth and pixel format
cam.AdcBitDepth.SetValue(PySpin.AdcBitDepth_Bit12)
if data_type == 'uint8':
    cam.PixelFormat.SetValue(PySpin.PixelFormat_Mono8)
if data_type == 'uint16':
    cam.PixelFormat.SetValue(PySpin.PixelFormat_Mono16)

#set output line
cam.LineSelector.SetValue(PySpin.LineSelector_Line1)
#cam.V3_3Enable.SetValue(False) # If using line 2 turn off 3.3V pullup resistor so the line can be used for output
#cam.LineMode.SetValue(PySpin.LineMode_Output) #if using line 2 set to output mode
cam.LineSource.SetValue(PySpin.LineSource_ExposureActive) #route desired output to Line 1 (try Counter0Active or ExposureActive)

#turn on chunk mode to get camera meta data
cam.ChunkModeActive.SetValue(True)
try:
    chunk_has_line_status = True
    cam.ChunkSelector.SetValue(PySpin.ChunkSelector_ExposureEndLineStatusAll)
    cam.ChunkEnable.SetValue(True)
except:  # noqa: E722
    chunk_has_line_status = False
try:
    cam.ChunkSelector.SetValue(PySpin.ChunkSelector_FrameID)
    cam.ChunkEnable.SetValue(True)
except:
    pass
    cam.ChunkSelector.SetValue(PySpin.ChunkSelector_Timestamp)
    cam.ChunkEnable.SetValue(True)


In [ ]:
#preview recording.
dev=ser.Serial(arduino_name)
dev.write(b'SET ALTERNATE_STATUS 1;')
app = vt.CameraApp(cam)
app.run()
app.close()
dev.write(b'SET ALTERNATE_STATUS 0;')
dev.close()


In [16]:
#Test Stimulator
dev=ser.Serial(arduino_name)
dev.write(b'SET STIM_STATUS 1;')
tm.sleep(10)
dev.write(b'SET STIM_STATUS 0;')
dev.close()


In [ ]:
#record.

dev=ser.Serial(arduino_name)

dev.write(b'SET ALTERNATE_STATUS 1;')

images,host_timestamps,dropped_frames,frame_ids,active_sequence,cam_timestamps = vt.grab_images(cam, node_map,length=length,data_type=data_type) #get video images and metadata

dev.write(b'SET ALTERNATE_STATUS 0;')

#this works for 16bit video right now
filename=output_path+prefix+'.tiff'
vt.write_tiff_stack(images,output_path,filename,data_type)


np.savetxt(os.path.join(output_path,prefix+'_'+barcode+'_metadata.csv'),
            np.transpose([host_timestamps,dropped_frames,frame_ids,active_sequence,cam_timestamps]),
            header ='host_timestamps, dropped_frames,frame_ids,active_sequence,cam_timestamps',
              delimiter=',')
